In [1]:
import lib.aoc.grid2d.grid as grid2d
import lib.aoc.grid2d.vector as vector2d
from heapq import heappop, heappush
from collections import defaultdict, deque

In [2]:
with open('../data/2024/day16.txt') as f:
    data = f.read()

In [3]:
def dijkstra_all_shortest(grid, start, goal):
    # We start at 'S' pointing east
    queue = [(0, start, vector2d.NESW[1])]

    # Lowest cost from the start for each (node,heading)
    cost_to_nodes = defaultdict(lambda : float('inf'))
    cost_to_nodes[(start, vector2d.NESW[1])] = 0

    # Shortest (node,heading) edges to each (node,heading)
    prev_nodes = defaultdict(set)

    while queue:
        cost, at, heading = heappop(queue)
        at_dir = (at, heading)

        # We can only go forward or turn 90 degrees
        sides = (v for v in vector2d.NESW if v != vector2d.invert(heading))

        # Check our neighbors
        for n_at, n_is in grid2d.neighbors(grid, at, sides):
            if n_is == '#': continue

            # It always costs +1 to move
            new_cost = cost_to_nodes[at_dir] + 1

            # If we're turning it also costs +1000
            slope = vector2d.slope(at, n_at)
            if slope != heading: new_cost += 1000

            n_at_dir = (n_at, slope)

            # If this is the new cheapest path we've found
            if new_cost < cost_to_nodes[n_at_dir]:
                cost_to_nodes[n_at_dir] = new_cost
                prev_nodes[n_at_dir] = {at_dir}
                heappush(queue, (new_cost, n_at, slope))
            # If this is an additional shortest path
            elif new_cost == cost_to_nodes[n_at_dir]:
                prev_nodes[n_at_dir].add(at_dir)

    # What is the shortest path leaving the goal?
    goal_v = min({v: cost_to_nodes[(goal, v)] for v in vector2d.NESW})

    # Keep track of shortest path tiles we've touched
    goal_tiles_touched = {(goal, goal_v)}
    queue = deque([(goal, goal_v)])

    # Rebuild the paths backwards from the goal
    while queue:
        at_heading = queue.pop()
        for n_at_heading in prev_nodes[at_heading]:
            if n_at_heading not in goal_tiles_touched:
                goal_tiles_touched.add(n_at_heading)
                queue.append(n_at_heading)

    # We only count tiles, not headings
    goal_tiles_touched = set([tile[0] for tile in goal_tiles_touched])

    return cost_to_nodes[(goal, goal_v)], len(goal_tiles_touched)

In [4]:
grid = grid2d.parse(data)
start = next(iter(grid2d.find_coords(grid, 'S')))
goal = next(iter(grid2d.find_coords(grid, 'E')))

part1, part2 = dijkstra_all_shortest(grid, start, goal)
print("Part 1:", part1)
print("Part 2:", part2)

Part 1: 93436
Part 2: 486
